# Application Manager

In [ ]:
from time import sleep
import requests
import ipywidgets as ipw
from os import path
import subprocess
from collections import OrderedDict
from IPython.display import clear_output

In [ ]:
# load registry
registry_url = 'https://raw.githubusercontent.com/materialscloud-org/mc-registry/master/apps.json'
registry = requests.get(registry_url).json()

In [ ]:
def get_appdir(name):
    return "/project/apps/"+name

In [ ]:
def update_app_list():
    selected = app_select.value
    app_select.value = None
    options = OrderedDict()
    for name in sorted(registry.keys()):
        label = name
        if path.exists(get_appdir(name)):
            label += " (installed)"
        options[label] = name
    app_select.options = options
    app_select.value = selected

In [ ]:
def on_app_change(c):
    with out_app:
        clear_output()
        if app_select.value:
            display_app_box(app_select.value)

app_select = ipw.Select()
update_app_list()
app_select.observe(on_app_change, names='value')

out_app = ipw.Output()
display(app_select, out_app)

In [ ]:
def display_app_box(name):
    print("Name: "+name)
    git_url = registry[name]['git_url']
    print("URL: "+git_url)
    meta_url = registry[name]['meta_url']
    metadata = requests.get(meta_url).json()
    print("Description: "+metadata['description'])
    
    if not path.exists(get_appdir(name)):
        btn = ipw.Button(description="install")
        btn.on_click(lambda b: install_app(name))   
        display(btn)
    else:
        ver_sel = mk_version_selector(name)
        btn = ipw.Button(description="remove")
        btn.on_click(lambda b: remove_app(name))
        display(ipw. HBox([btn, ver_sel]))

    btn.disabled = name == "home"

In [ ]:
def mk_version_selector(name):
    git_url = registry[name]['git_url']
    appdir = get_appdir(name)
    ! git -C $appdir fetch --prune --quiet
    
    
    ref_lines = ! git -C $appdir show-ref
    options = {}
    for line in ref_lines:
        commit, ref = line.split()
        options[ref] = commit

    head_lines = ! git -C $appdir rev-parse HEAD
    head = head_lines[0]
    if head not in options.values():
        options[head] = head
    print("head:"+head)
    print("appdir: "+appdir)
    ref_drop = ipw.Dropdown(description="Version:", options=options, value=head)
    def on_ref_change(c):
        change_ref(name, ref_drop.value)
    ref_drop.observe(on_ref_change, names='value')
    
    return(ref_drop)

In [ ]:
def change_ref(name, commit):
    print("Changing version of app %s..."%name)
    appdir = get_appdir(name)
    ! git -C $appdir checkout --quiet $commit
    sleep(1)
    update_app_list()

In [ ]:
def install_app(name):
    print("Installing app %s..."%name)
    assert name != "home"
    git_url = registry[name]["git_url"]
    appdir = get_appdir(name)
    ! git clone --quiet $git_url $appdir
    #TODO maybe call "jupyter trust" on all notebooks
    sleep(1)
    update_app_list()

In [ ]:
def remove_app(name):
    print("Removing app %s..."%name)
    assert name != "home"
    appdir = get_appdir(name)
    ! rm -rf $appdir
    sleep(1)
    update_app_list()

In [ ]:
%%html
<a href="./start.ipynb"><button>Return to Launcher</button></a>